In [ ]:
!pip install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git
!pip install -q torchviz
!pip install albumentations==0.4.5

In [ ]:
import time
import skimage.io
import numpy as np
import pandas as pd
import cv2
import PIL.Image
from pylab import rcParams
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from warmup_scheduler import GradualWarmupScheduler
import albumentations
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score
from tqdm import tqdm_notebook as tqdm
from torchvision.models import resnet34

In [ ]:
data_path = '../input/prostate-cancer-grade-assessment/'
df_train = pd.read_csv(data_path + 'train.csv')
image_path = '../input/panda_data/'
fold = 0
num_workers = 4
init_lr = 3e-5
warmup_factor = 3
warmup_epo = 1
n_epochs = 40
batch_size = 8
num_folds = 5
device = torch.device('cuda')

In [ ]:
skf = StratifiedKFold(num_folds, shuffle=True, random_state=42)
df_train['fold'] = -1
for i, (train_idx, valid_idx) in enumerate(skf.split(df_train, df_train['isup_grade'])):
    df_train.loc[valid_idx, 'fold'] = i
df_train.head()

In [ ]:
class Data_Loader(Dataset):
    def __init__(self,
                 df,
                 rand=False,
                 transform=None,
                ):

        self.df = df.reset_index(drop=True)
        self.rand = rand
        self.transform = transform

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_id = row.image_id
        folder_idx = row.folder_idx
        if(folder_idx==0 or folder_idx == 5):
            images = plt.imread((image_path + str(img_id) + '.png'))
        else:
            images = plt.imread((image_path + str(img_id) + '.png'))
        images = cv2.cvtColor(images, cv2.COLOR_BGRA2BGR)
        images = 1 - images
        images = images.astype(np.float32)
        images = images.transpose(2, 0, 1)

        label = np.zeros(5).astype(np.float32)
        label[:row.isup_grade] = 1.
        return torch.tensor(images), torch.tensor(label)

In [ ]:
dataset_show = Data_Loader(df_train)
rcParams['figure.figsize'] = 20,10
for i in range(2):
    f, axarr = plt.subplots(1,5)
    for p in range(5):
        idx = np.random.randint(0, len(dataset_show))
        img, label = dataset_show[idx]
        label = label.numpy().astype(int)
        axarr[p].imshow(1. - img.transpose(0, 1).transpose(1,2).squeeze())
        axarr[p].set_title('i_sup grade : ' + str(np.sum(label)))

In [ ]:
criterion = nn.BCEWithLogitsLoss()

class ResNetDetector(nn.Module):
    def __init__(self):
        super(ResNetDetector, self).__init__()

        self.softmax = nn.Softmax(dim=1)
        self.dense_1 = nn.Linear(512, 5)
        self.resnet = resnet34(pretrained=True)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])
        
    def forward(self, img):
        feat = self.resnet(img).squeeze()

        isup_logit = self.dense_1(feat)
        return isup_logit

model = ResNetDetector()

In [ ]:
def engine(train_loader, valid_loader, optimizer):
    train_loss = []
    val_loss = []
    PREDS = []
    TARGETS = []

    model.train()
    train_loss = []
    bar = tqdm(train_loader)
    for (data, target) in bar:
        
        data, target = data.to(device), target.to(device)
        loss_func = criterion
        optimizer.zero_grad()
        logits = model(data)
        loss = loss_func(logits, target)
        loss.backward()
        optimizer.step()

        loss_np = loss.detach().cpu().numpy()
        train_loss.append(loss_np)
        smooth_loss = sum(train_loss[-100:]) / min(len(train_loss), 100)
        total_loss = sum(train_loss) / max(len(train_loss), 1)
        bar.set_description('loss: %.5f, smth: %.5f, total: %.5f' % (loss_np, smooth_loss, total_loss))
    model.eval()
    with torch.no_grad():
        for (data, target) in tqdm(loader):
            data, target = data.to(device), target.to(device)
            logits = model(data)

            loss = criterion(logits, target)

            pred = logits.sigmoid().sum(1).detach().round()
            PREDS.append(pred)
            TARGETS.append(target.sum(1))
            val_loss.append(loss.detach().cpu().numpy())
        val_loss = np.mean(val_loss)

    PREDS = torch.cat(PREDS).cpu().numpy()
    TARGETS = torch.cat(TARGETS).cpu().numpy()
    acc = (PREDS == TARGETS).mean() * 100.
    
    qwk = cohen_kappa_score(PREDS, TARGETS, weights='quadratic')
    print('qwk', qwk)

    return train_loss, val_loss, acc, qwk 

In [ ]:
train_idx = np.where((df_train['fold'] != fold))[0]
valid_idx = np.where((df_train['fold'] == fold))[0]

df_this  = df_train.loc[train_idx]
df_valid = df_train.loc[valid_idx]

dataset_train = Data_Loader(df_this)
dataset_valid = Data_Loader(df_valid)

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, sampler=RandomSampler(dataset_train), num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, sampler=SequentialSampler(dataset_valid), num_workers=num_workers)

model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=init_lr/warmup_factor)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs-warmup_epo)
scheduler = GradualWarmupScheduler(optimizer, multiplier=warmup_factor, total_epoch=warmup_epo, after_scheduler=scheduler_cosine)

print(len(dataset_train), len(dataset_valid))

In [ ]:
qwk_max = 0.
for epoch in range(1, n_epochs+1):
    print(time.ctime(), 'Epoch:', epoch)
    scheduler.step(epoch-1)
    train_loss, val_loss, acc, qwk = engine(train_loader, valid_loader, optimizer)
    content = time.ctime() + ' ' + f'Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {np.mean(train_loss):.5f}, val loss: {np.mean(val_loss):.5f}, acc: {(acc):.5f}, qwk: {(qwk):.5f}'
    print(content)
    with open(f'log_{kernel_type}.txt', 'a') as appender:
        appender.write(content + '\n')
    if qwk > qwk_max:
        torch.save(model.state_dict(), f'{kernel_type}.pth')
        qwk_max = qwk
torch.save(model.state_dict(), os.path.join(f'{kernel_type}_final_fold{fold}.pth'))